## Use this file to test the energyData.sqlite database

In [61]:
# import necessary libraries
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine, inspect

### Create a connection with the database file

In [62]:
engine = create_engine("sqlite:///assets/db/energyData.sqlite")

### Use this path to interact with the database created using SQLAlchemy

In [63]:
# reflect an existing database into a new model
Base = automap_base()

In [64]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [65]:
# Print all of the classes mapped to the Base
print(Base.classes.keys())

['StateEnergyConsumptionSector']


In [66]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [67]:
# Collect the names of tables within the database
inspector.get_table_names()

['StateEnergyConsumptionSector', 'sqlite_sequence']

In [68]:
# Query All Records in the the Database
data = engine.execute("SELECT * FROM StateEnergyConsumptionSector")
for record in data:
    print(record)

(1, "b'Illinois'", 891.6, 796.2, 1176.2, 1007.5)
(2, "b'Kentucky'", 332, 257, 598.5, 470.7)
(3, "b'Texas'", 1640.6, 1600.5, 6754.4, 3370.3)


In [69]:
# Create a "Metadata" Layer That Abstracts our SQL Database
Base.metadata.create_all(engine)

# Use this to clear out the db
# Base.metadata.drop_all(engine)

In [70]:
# Save a reference to the StateEnergyConumptionSector table as `ConsumptionSector`
ConsumptionSector = Base.classes.StateEnergyConsumptionSector

In [87]:
# Create a database session object
session = Session(engine)

In [88]:
#results = session.query(ConsumptionSector.State).all()
#print(results)

In [94]:
def consumptionSectors():
    # Query all states
    results = session.query(ConsumptionSector.State, 
            ConsumptionSector.Residential, 
            ConsumptionSector.Commercial, 
            ConsumptionSector.Industrial,
            ConsumptionSector.Transportation).all()
    #print(results)
    # Create a dictionary from the row data and append to a list of all_passengers
    consumption_sector = []
    for State, Residential, Commercial, Industrial, Transportation in results:
        consumption_sector_dict = {}
        consumption_sector_dict["State"] = State
        consumption_sector_dict["Residential"] = Residential
        consumption_sector_dict["Commercial"] = Commercial
        consumption_sector_dict["Industrial"] = Industrial
        consumption_sector_dict["Transportation"] = Transportation
        consumption_sector.append(consumption_sector_dict)
    #return jsonify(consumption_sector)
    print(consumption_sector)

consumptionSectors()

[{'State': "b'Illinois'", 'Residential': 891.6, 'Commercial': 796.2, 'Industrial': 1176.2, 'Transportation': 1007.5}, {'State': "b'Kentucky'", 'Residential': 332, 'Commercial': 257, 'Industrial': 598.5, 'Transportation': 470.7}, {'State': "b'Texas'", 'Residential': 1640.6, 'Commercial': 1600.5, 'Industrial': 6754.4, 'Transportation': 3370.3}]


In [80]:
# Create a dictionary from the row data and append to a list of all_passengers
consumption_sector = []
for State, Residential, Commercial, Industrial, Transportation in results:
    consumption_sector_dict = {}
    consumption_sector_dict["State"] = State
    consumption_sector_dict["Residential"] = Residential
    consumption_sector_dict["Commercial"] = Commercial
    consumption_sector_dict["Industrial"] = Industrial
    consumption_sector_dict["Transportation"] = Transportation
    consumption_sector.append(consumption_sector_dict)
    return jsonify(consumption_sector)

SyntaxError: 'return' outside function (<ipython-input-80-762afa52a483>, line 11)

### Use this path to interact with the database created by Pandas

In [134]:
# Query All Records in the the Database
data = engine.execute("SELECT * FROM energyconsumpsector")
for record in data:
    print(record)

(0, 'Illinois', 891.6, 796.2, 1176.2, 1007.5)
(1, 'Kentucky', 332.0, 257.0, 598.5, 470.7)
(2, 'Texas', 1640.6, 1600.5, 6754.4, 3370.3)


In [135]:
# Query All Records in the the Database
data = engine.execute("SELECT * FROM electricitygeneration")
for record in data:
    print(record)

(0, 'Illinois', 4, 985, 4543, 7641, 0, 1255)
(1, 'Kentucky', 8, 877, 4353, 0, 316, 39)
(2, 'Texas', 4, 15251, 7450, 3430, 150, 6962)


In [169]:
import sqlite3
import json
con = sqlite3.connect('./assets/db/energyData.sqlite')
#con.row_factory = sqlite3.Row
c = con.cursor()
result = c.execute('select * from energyconsumpsector')
items = [dict(zip([key[0] for key in c.description],row))for row in result]
print(json.dumps({'EnergyConsumptionSector':items}))

# items = []
# for row in result:
#         for key in c.description:
#             items.append({key[0]: value for value in row})
# print(json.dumps({'items':items}))

# for row in c.execute('select * from energyconsumpsector'):
#     print(row.keys())
#     print(tuple(row))
#     data = c.fetchall()
# print(json.dumps(data))

con.close()

{"EnergyConsumptionSector": [{"index": 0, "State": "Illinois", "Residential": 891.6, "Commercial": 796.2, "Industrial": 1176.2, "Transportation": 1007.5}, {"index": 1, "State": "Kentucky", "Residential": 332.0, "Commercial": 257.0, "Industrial": 598.5, "Transportation": 470.7}, {"index": 2, "State": "Texas", "Residential": 1640.6, "Commercial": 1600.5, "Industrial": 6754.4, "Transportation": 3370.3}]}
[{'index': 0, 'State': 'Illinois', 'Residential': 891.6, 'Commercial': 796.2, 'Industrial': 1176.2, 'Transportation': 1007.5}, {'index': 1, 'State': 'Kentucky', 'Residential': 332.0, 'Commercial': 257.0, 'Industrial': 598.5, 'Transportation': 470.7}, {'index': 2, 'State': 'Texas', 'Residential': 1640.6, 'Commercial': 1600.5, 'Industrial': 6754.4, 'Transportation': 3370.3}]


In [153]:
import json
import sqlite3

def get_my_jsonified_data(key):
    with sqlite3.connect('./assets/db/energyData.sqlite') as conn:
        cursor = conn.cursor()
        cursor.execute("select * from energyconsumpsector")
        data = cursor.fetchall()
        return json.dumps(data)

print(data)
#https://stackoverflow.com/questions/25564399/send-json-response-from-sqlite-queries-in-python